# Classification and Regression, Part 2 - Exercise Solution

In [1]:
import numpy as np
import seaborn as sns

# Exercise 1. Univariate Classification

---

### About the data

---
The dataset you will use for today exercise is a modified version of Computers https://www.timeseriesclassification.com/description.php?Dataset=Computers

These problems were taken from data recorded as part of government sponsored study called Powering the Nation. The intention was to collect behavioural data about how consumers use electricity within the home to help reduce the UK's carbon footprint. The data contains readings from 251 households, sampled in two-minute intervals over a month. Each series is length 720 (24 hours of readings taken every 2 minutes). Classes are Desktop and Laptop.



---

### Read the data

Load the numpy files X_train_computers.npy, X_test_computers.npy, y_train_computers.npy, y_test_computers.npy


In [2]:
X_train = np.load('data/X_train_computers.npy')
X_test = np.load('data/X_test_computers.npy')
y_train = np.load('data/y_train_computers.npy')
y_test = np.load('data/y_test_computers.npy')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((251, 1, 720), (250, 1, 720), (251,), (250,))

In [3]:
from sktime.transformations.series.impute import Imputer

In [4]:
X_train_no_out = X_train.copy()
X_test_no_out = X_test.copy()

In [5]:
# treat outliers as missing values
X_train_no_out[X_train > 8000] = np.nan
X_test_no_out[X_test > 8000] = np.nan

In [6]:
# replace missing values
imputer = Imputer(method="drift")

In [7]:
# the imputer is instance-wise, so it is ok to fit_transform also the test set (there is no data leakage)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.fit_transform(X_test)

---

## Classification
After you removed the main problems in the data, you can start the classification process.
Does the dataset need further preprocessing (normalization maybe)? If so, do it. Use the LabelEncoder to transform the target variable in a numerical format.

### Goal
Your goal here is to find the best possible classification pipeline to predict the class of the computers dataset. You can use any classifier you saw in the course up to now.

### Hyperparameter tuning
Try to find the best hyperparameters for your model. You can a validation set, cross validation or a GridSearchCV or RandomizedSearchCV for this task.

### Evaluation
Don't overfit the test set! Only once you think you found the best model, test performance on the test set. Find the model that has the highest ROC AUC score.

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

### Label Encoding

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
le = LabelEncoder()

In [11]:
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

### How does a dummy classifier perform?

In [12]:
from sktime.classification.dummy import DummyClassifier

In [13]:
dummy = DummyClassifier()

In [14]:
cross_val_score(dummy, X_train_imputed, y_train_encoded, cv=5, scoring="roc_auc").mean()

0.5

### Normalization

In [15]:
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = TabularToSeriesAdaptor(StandardScaler(), fit_in_transform=True)

### Weasel

In [17]:
from sktime.classification.dictionary_based import WEASEL

In [18]:
weasel = scaler * WEASEL(support_probabilities=True)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [19]:
%%time
cross_val_score(weasel, X_train_imputed, y_train_encoded, cv=5, scoring="roc_auc").mean()

CPU times: user 53.9 s, sys: 42.2 s, total: 1min 36s
Wall time: 37.6 s


0.7164061538461539

### MLP

In [20]:
from sktime.transformations.panel.reduce import Tabularizer
from sklearn.neural_network import MLPClassifier

In [21]:
mlp = scaler * Tabularizer() * MLPClassifier(random_state=0)

In [22]:
%%time
cross_val_score(mlp, X_train_imputed, y_train_encoded, cv=5, scoring="roc_auc").mean()

CPU times: user 14.7 s, sys: 10.7 s, total: 25.4 s
Wall time: 4.54 s


0.528676923076923

### CNN

In [23]:
from sktime.classification.deep_learning import CNNClassifier

In [24]:
cnn = scaler * CNNClassifier(n_conv_layers=1, random_state=0, verbose=True, n_epochs=100)

In [25]:
%%time
cross_val_score(cnn, X_train_imputed, y_train_encoded, cv=5, scoring="roc_auc").mean()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 720, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 714, 6)         │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d               │ (None, 238, 6)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1428)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         2,858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,906 (11.35 KB)

 Trainable params: 2,906 (11.35 KB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 2,906 (11.35 KB)

 Trainable params: 2,906 (11.35 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4973 - loss: 1.0043  
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4858 - loss: 0.7786 
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5967 - loss: 0.6483
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7275 - loss: 0.5650 
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7350 - loss: 0.5261 
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7639 - loss: 0.4991 
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7845 - loss: 0.4872 
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7737 - loss: 0.4738 
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7705 - loss: 0.4635 
Epoch 10/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7831 - loss: 0.4538 
Epoch 11/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7831 - loss: 0.4453 
Epoch 12/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

0.584443076923077

### Rocket

In [26]:
from sktime.classification.kernel_based import RocketClassifier

In [27]:
rocket = scaler * RocketClassifier(random_state=0, rocket_transform="rocket")

In [28]:
%%time
cross_val_score(rocket, X_train_imputed, y_train_encoded, cv=5, scoring="roc_auc").mean()

CPU times: user 3min 4s, sys: 14.7 s, total: 3min 19s
Wall time: 3min 3s


0.762

#### Finetuning

In [29]:
from sktime.transformations.compose import OptionalPassthrough

In [30]:
pipe = OptionalPassthrough(scaler) * RocketClassifier(random_state=0)

In [31]:
param_grid = {
    "optionalpassthrough__passthrough": [True, False],
    "rocketclassifier__rocket_transform": ['rocket', 'minirocket'],
}

In [32]:
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring="roc_auc", verbose=True, n_jobs=-1)

In [33]:
%%time
grid_search.fit(X_train_imputed, y_train_encoded)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


CPU times: user 33.6 s, sys: 492 ms, total: 34.1 s
Wall time: 2min 47s


GridSearchCV(cv=5,
             estimator=ClassifierPipeline(classifier=RocketClassifier(random_state=0),
                   transformers=[OptionalPassthrough(transformer=TabularToSeriesAdaptor(fit_in_transform=True,
                                                                                        transformer=StandardScaler()))]),
             n_jobs=-1,
             param_grid={'optionalpassthrough__passthrough': [True, False],
                         'rocketclassifier__rocket_transform': ['rocket',
                                                                'minirocket']},
             scoring='roc_auc', verbose=True)

In [34]:
grid_search.best_params_

{'optionalpassthrough__passthrough': True,
 'rocketclassifier__rocket_transform': 'rocket'}

In [35]:
grid_search.best_estimator_

ClassifierPipeline(classifier=RocketClassifier(random_state=0),
                   transformers=[OptionalPassthrough(transformer=TabularToSeriesAdaptor(fit_in_transform=True,
                                                                                        transformer=StandardScaler()))])

In [36]:
grid_search.best_score_

0.762

In [37]:
grid_search.best_estimator_.fit(X_train_imputed, y_train_encoded)

ClassifierPipeline(classifier=RocketClassifier(random_state=0),
                   transformers=[OptionalPassthrough(transformer=TabularToSeriesAdaptor(fit_in_transform=True,
                                                                                        transformer=StandardScaler()))])

In [38]:
y_pred = grid_search.best_estimator_.predict(X_test_imputed)

In [39]:
roc_auc_score(y_test_encoded, y_pred)

0.772

# Exercise 2. Multivariate Regression

---

### About the data

The goal of this dataset is to predict total energy usage in kWh of a house. This dataset contains 138 time series obtained from the Appliances Energy Prediction dataset from the UCI repository. The time series has 24 dimensions. This includes temperature and humidity measurements of 9 rooms in a house, monitored with a ZigBee wireless sensor network. It also includes weather and climate data such as temperature, pressure, humidity, wind speed, visibility and dewpoint measured from Chievres airport. The data set is averaged for 10 minutes period and spanning 4.5 months.

### Load the data

Load the numpy files X_train_AppliancesEnergy.npy, X_test_AppliancesEnergy.npy, y_train_AppliancesEnergy.npy, y_test_AppliancesEnergy.npy

In [40]:
X_train = np.load('data/X_train_AppliancesEnergy.npy')
X_test = np.load('data/X_test_AppliancesEnergy.npy')
y_train = np.load('data/y_train_AppliancesEnergy.npy')
y_test = np.load('data/y_test_AppliancesEnergy.npy')
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((95, 24, 144), (42, 24, 144), (95,), (42,))

## Regression
Similar to the previous exercise, you can use any regressor you saw in the course up to now. Find the best model in terms of MAE. As metric use `neg_mean_absolute_error` in the cross_val_score function (higher is better).

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

### Dummy Regressor

In [42]:
from sktime.regression.dummy import DummyRegressor

In [43]:
dummy = DummyRegressor()

In [44]:
cross_val_score(dummy, X_train, y_train, cv=5, scoring="neg_mean_absolute_error").mean()

-3.537271468144044

### CNN

In [45]:
from sktime.regression.deep_learning import CNNRegressor

In [46]:
cnn = scaler * CNNRegressor(n_conv_layers=2, random_state=0, verbose=True, n_epochs=500)

In [47]:
%%time
cross_val_score(cnn, X_train, y_train, cv=5, scoring="neg_mean_absolute_error").mean()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 144, 24)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 138, 6)         │         1,014 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_5             │ (None, 46, 6)          │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 40, 12)         │           516 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_6             │ (None, 13, 12)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 156)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           157 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,687 (6.59 KB)

 Trainable params: 1,687 (6.59 KB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 1,687 (6.59 KB)

 Trainable params: 1,687 (6.59 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 123.1175  
Epoch 2/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 34.8061 
Epoch 3/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 24.3526 
Epoch 4/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 15.8473 
Epoch 5/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 12.3894 
Epoch 6/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 8.6457  
Epoch 7/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 7.1204 
Epoch 8/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 5.0574 
Epoch 9/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 4.3250 
Epoch 10/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 3.3308 
Epoch 11/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 2.8073 
Epoch 12/500
5/5 ━━━━━

-3.420548584887856

### Rocket

In [48]:
from sktime.regression.kernel_based import RocketRegressor

In [49]:
rocket = scaler * RocketRegressor(random_state=0, rocket_transform="rocket")

In [50]:
%%time
cross_val_score(rocket, X_train, y_train, cv=5, scoring="neg_mean_absolute_error").mean()

CPU times: user 1min 17s, sys: 7.47 s, total: 1min 25s
Wall time: 1min 13s


-2.4652168913156585

#### Finetuning

In [51]:
from sktime.transformations.compose import OptionalPassthrough
from sktime.transformations.compose import FeatureUnion
from sktime.transformations.series.difference import Differencer

In [52]:
pipe = OptionalPassthrough(scaler) * RocketRegressor(random_state=0)

In [53]:
param_grid = {
    "optionalpassthrough__passthrough": [True, False],
    "rocketclassifier__rocket_transform": ['rocket', 'minirocket'],
}

In [54]:
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_absolute_error", verbose=True, n_jobs=-1)

In [55]:
%%time
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
CPU times: user 13.4 s, sys: 186 ms, total: 13.6 s
Wall time: 1min 5s


GridSearchCV(cv=5,
             estimator=RegressorPipeline(regressor=RocketRegressor(random_state=0),
                  transformers=[OptionalPassthrough(transformer=TabularToSeriesAdaptor(fit_in_transform=True,
                                                                                       transformer=StandardScaler()))]),
             n_jobs=-1,
             param_grid={'optionalpassthrough__passthrough': [True, False],
                         'rocketclassifier__rocket_transform': ['rocket',
                                                                'minirocket']},
             scoring='neg_mean_absolute_error', verbose=True)

In [56]:
grid_search.best_params_

{'optionalpassthrough__passthrough': True,
 'rocketclassifier__rocket_transform': 'rocket'}

In [57]:
grid_search.best_estimator_

RegressorPipeline(regressor=RocketRegressor(random_state=0),
                  transformers=[OptionalPassthrough(transformer=TabularToSeriesAdaptor(fit_in_transform=True,
                                                                                       transformer=StandardScaler()))])

In [58]:
grid_search.best_score_

-2.4652168913156585

In [59]:
grid_search.best_estimator_.fit(X_train, y_train)

RegressorPipeline(regressor=RocketRegressor(random_state=0),
                  transformers=[OptionalPassthrough(transformer=TabularToSeriesAdaptor(fit_in_transform=True,
                                                                                       transformer=StandardScaler()))])

In [60]:
y_pred = grid_search.best_estimator_.predict(X_test)

In [62]:
mean_absolute_error(y_test, y_pred)

1.80104826138604